In [630]:
import glob
files = glob.glob('D:/data/毒品/單被告判決書_3/*')
len(files),files[:2]

(17044,
 ['D:/data/毒品/單被告判決書_3\\福建金門地方法院,101,訴,2,20120425,1.txt',
  'D:/data/毒品/單被告判決書_3\\福建金門地方法院,101,訴,24,20130206,1.txt'])

In [631]:
import re
import random
%run tool.ipynb
print(tool.__doc__)


    maintent ,add_table ,one_accused ,one_accused_txt ,digit_maker,get_digits ,drug_cats ,re_trade_para, re_sell_drug2,
    word_location,re_search_digit,
    


# main

In [653]:
%run tool.ipynb
#main
#a,b,c,d,e,g=[],[],[],[],[],[]
cats = drug_cats()
data =[]
skip = ['扣案','扣得','賺','惟被告','四捨五入','不詳數量','伺機','牟利之意圖','牟利意圖','意圖牟利','被告作價',
        '第三級','愷他命','海洛因','有期徒刑','罰金','元以上'
        ]
for i,file in enumerate(files):
    '''
    if i < 3000:
        continue
    if i > 4000:
        break
    '''
    if i % 1700 == 0:
        print(i,end=',')
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    #txt1 = txt.replace('\n\n','\n')
    txt2 = txt.replace('\n','').replace('\u3000','').replace(' ','').replace(',','')
    content = '書記官'.join(txt2.split('書記官')[:-1])
    name = one_accused_txt(txt)
    onoff = 'no'
    #初步排除無元、公克、二級毒品名
    if '元' in txt2 and '公克' in txt2:
        for cat in cats:
            if cat in txt2:
                #g1.append(file),13070
                onoff = 'yes'
                break
    if onoff == 'no':
        data.append((file,'','','','無完整販賣資訊'))
        continue
        #g2.append(file),3974
    elif '\ntable\n' in txt:
        table = txt.split('\ntable\n')[1]
        m1 = re_trade_para(table,name)
        if m1 != []: 
            pass
            #a.append(file),1422
        else:
            m1 = re_trade_para(content,name)
            if m1 != []: 
                pass
                #b.append(file),2213
            elif re_sell_drug2(txt2,name) != None: #另外設計找出表內的規則
                cases = table.split('###編號:')[1:]
                onoff = 'm1_process'
                for case in cases:
                    if '元' in case and '公克' in case:
                        for cat in cats:
                            if cat in case: #
                                #c.append(file),1676
                                mark = '人工優先表格'
                                data.append((file,'','','',mark))
                                onoff = 'm0_process'
                                break
                    if onoff == 'm0_process':
                        break
                if onoff == 'm1_process':
                    m1 = re_trade_para(content,name)
                    if m1 != []:
                        pass
                        #d.append(file),0
                    else: #該判決書無完整交易資料
                        pass
                        #g.append(file)
                    
    else:
        m1 = re_trade_para(content,name)
        if m1 != []:
            pass
            #e.append(file),1542
        else: #該判決書無完整交易資料
            pass
            #g.append(file),5262
    #rowdata = file,品名drugs,總重weis,總價dols,remark
    #m0_process:
    if onoff == 'm0_process':
        pass
    #m1_process:
    if m1 == []:
        data.append((file,'','','','低度可能有資訊') )
        continue
    #處理判決書裡每個篩出的段落
    for m1_ in m1:
        row = []
        mark =''
        #不在同一段話
        onoff = 'off'
        if '販賣' in m1_:
            for w in m1_.split('。'):
                if '販賣' in w and '元' in w and '公克' in w:
                    onoff = 'on'
            if onoff == 'off':
                data.append((file,'','','','中度可能有資訊') )
                continue
        #毒品名
        onoff = 'off'
        for cat in cats:
            if cat in m1_:
                drug = cat
                onoff= 'on'
                break
        if onoff == 'off':
            #data.append((file,'','','') )
            continue
        #評論句
        if re.search('被告\w{0,7}不可能.{0,25}販賣',m1_) != None:
            #data.append((file,'','','') )
            continue
        #m1_開頭有金額或公克數被截掉而修正
        regex = ['\d*|,\d*元|\d*,\d*元|\d*公克|,\d*公克|\d*,\d*公克|.\d*公克|\d*.\d*公克|\d*,\d*.\d*公克',
                 '[零壹貳參肆伍陸柒捌玖一二三四五六七八九十拾百佰千仟萬億兆]*[元公克]{1,2}'
                ]
        regex = '|'.join(regex)
        sea = re.search(regex,m1_)
        if sea != None and sea.start() == 0:
            m1_ = m1_[sea.end():]
        #save
        if '依序' in m1_:
            weis = get_digits(m1_,'公克')
            dols = get_digits(m1_)
            if len(weis) == len(dols):
                for w in range(len(weis)):
                    if mark == '':
                        mark = '低度需驗證'
                    row = (file,drug,weis[w],dols[w],mark)
                    data.append(row)
                continue
        #1.file,2.drug        
        row = [file,drug]
        #3.wei
        weis = get_digits(m1_,'公克')
        if len(weis) == 0:
            continue
        #row.append(weis)
        if len(weis) > 1: #距離元最近的公克數
            loc = word_location('公克',m1_)
            loc = loc[:len(weis)]
            minus = m1_.find('元')
            new =[]
            for w in loc:
                new.append(abs(w-minus))
            for it,num in enumerate(new):
                if num == min(new):
                    try:
                        wei = weis[it]
                        if mark == '':
                            mark = '低度需驗證'
                    except IndexError:
                        wei = weis[it-1]
                        if mark == '' or mark == '低度需驗證':
                            mark = '中度需驗證'
                    row.append(wei)
                    break
        else:
            wei = weis[0]
            row.append(wei)
        #4.dollar
        dols = get_digits(m1_)
        sea = re.search('\w*元\w*販[賣售]\w*|\w*,\w*元\w*販[賣售]\w*',m1_)
        if len(dols) == 0:
            continue
        elif len(dols) == 1 and '同時販賣' in m1_:
            if mark == '' or mark == '低度需驗證':
                mark = '中度需驗證'
            data.append((file,'','','',mark))
            continue
        elif len(dols) == 1: 
            dol = int(dols[0])
        elif sea != None: #取與販賣在同一句的金額:
            try:
                dol = int(get_digits(sea.group())[0])
                if mark == '':
                    mark = '低度需驗證'
            except IndexError:
                dol = int(dols[0])
                if mark == '' or mark == '低度需驗證' :
                    mark = '中度需驗證'
        else:
            if mark == '' or mark == '低度需驗證' :
                mark = '中度需驗證'
            prices = [int(x) for x in get_digits(m1_)]
            prices.sort()
            dol = prices[-1]
        if re.search('(\w\w\w){}'.format(dol),m1_) != None:
            if re.search('(\w\w\w){}'.format(dol),m1_).group(1) == '每公克':
                if mark == '' or mark == '低度需驗證' :
                    mark = '中度需驗證'
                dol = int(float(dol)*float(wei))
        if dol < 200:
            continue
        row.append(dol)
        #skip
        onoff = 'on'
        for w in skip:
            if w in m1_:
                onoff = 'off'
        if onoff == 'off':
            if mark == '' or mark == '低度需驗證' or mark == '中度需驗證':
                mark = '高度需驗證'
            data.append((file,'','','',mark))
            continue
        row.append(mark)
        data.append(tuple(row) )


0,1700,3400,5100,6800,8500,10200,11900,13600,15300,17000,

In [654]:
print(len(data))
len({x[0] for x in data})

22399


15480

In [655]:
#該份判決書有m1但最終無存出資料，列為中度可能有資訊
data2 = data
d0 = {x[0] for x in data2}
for file in files:
    if file not in d0:
        data2.append((file,'','','','中度可能有資訊'))
len(data2)

23963

In [657]:
import pandas as pd
name = ['path','drug','weight','dol','remark']
df = pd.DataFrame(data = data,columns=name)
df.head()
path = r'D:\Google雲端硬碟\毒品code\drug2_trades_3.csv'
df.to_csv(path,encoding='utf-8-sig',index=False)
path = r'D:\Google雲端硬碟\毒品code\drug2_trades_3.xlsx'
df.to_excel(path,encoding='utf-8-sig',index=False)
#數值有小錯誤，再經人工修改xlsx後，轉存ver2

# --------------------------------------------------------------------

In [556]:
for i in data:
    print(i)
for i in m1:
    print(i)

('D:/data/毒品/單被告判決書_3\\臺灣新北地方法院,101,訴,1112,20121214,1.txt', '安非他命', ['1', '1'], '1', '1', 3000)
段160巷21號5樓住處樓下，將重量約1公克之第二級毒品甲基安非他命交付予林鴻祥，林鴻祥即交付現金新臺幣（下同）3000元予莊雅雯，以此方式販賣第二級毒品予林鴻祥1次以營利。
牟利之行為，於偵查中亦供稱：（檢察官問：你販賣毒品獲利多少？）我進價1公克安非他命進價2500元，賣出3000元，每1公克安非他命原則上可獲利500元等


In [510]:
%run tool.ipynb
i,file,m1_,get_digits(m1_,'公克'),get_digits(m1_)

('即販賣第二級毒品予買家陳桂煌部分，此次被告販賣之數量已達17公克，約定之交易價格為1萬1000元，又被告販賣第二級毒品予陳桂煌',
 'D:/data/毒品/單被告判決書_3\\臺灣新北地方法院,107,訴,1140,20190618,2.txt',
 '即販賣第二級毒品予買家陳桂煌部分，此次被告販賣之數量已達17公克，約定之交易價格為1萬1000元，又被告販賣第二級毒品予陳桂煌',
 ['17'],
 ['11000'])

In [105]:
file = 'D:/data/毒品/單被告判決書_3\\福建金門地方法院,106,訴,34,20180503,4.txt'
with open(file,encoding='utf-8-sig') as f:
    txt =f.read()
txt1 = txt.replace('\n\n','\n')
print(txt1)

字號:KMDM,106,訴,34,20180503,4
判案原由:違反毒品危害防制條例等
內文:
福建金門地方法院刑事判決　　　　　　　 106年度訴字第34號
公　訴　人　福建金門地方法院檢察署檢察官
被　　　告　劉致瑄
訴訟代理人　吳忠德律師 （法律扶助）
上列被告因違反毒品危害防制條例等案件，經檢察官提起公訴（
106年度偵字第648、720、721、722、729、733、747號、106年
度少連偵字第6號），本院判決如下：
    主  文
甲○○犯如附表一、二所示之各罪，各處如附表一、二所示之宣
告刑。如附表一及如附表二編號二至七所示不得易科罰金之罪共
拾柒罪，應執行有期徒刑肆年拾月。如附表二編號一所示之罪，
處有期徒刑肆月，如易科罰金，以新臺幣壹仟元折算壹日。
未扣案之販賣毒品所得新臺幣壹萬柒仟元、行動電話（含門號○
○○○○○○○○○號SIM卡壹張）壹具均沒收，如全部或一部
不能沒收或不宜執行沒收時，追徵其價額。
    事  實
一、甲○○明知甲基安非他命屬毒品危害防制條例所列管之第二
    級毒品，亦為衛生福利部公告查禁之禁藥，未經許可不得販
    賣及轉讓，竟分別為下列行為：
  甲○○基於販賣第二級毒品甲基安非他命以營利之犯意，於
    如附表一所示之時間、地點、方式，販賣如附表一所示之甲
    基安非他命與張○○2次、楊○○1次、李○○2次、黃○○1
    次。
  甲○○為成年人，明知甲基安非他命為毒品危害防制條例第
    2條第2項第2款所列之第二級毒品，依法不得轉讓、持有，
    且知林○○為未成年人（民國○年○月生，真實姓名年籍資
    料詳卷），竟基於轉讓第二級毒品甲基安非他命之犯意，於
    如附表二編號1所示之時間、地點、方式，無償轉讓甲基安
    非他命與少年林○○1次。
  甲○○明知甲基安非他命為毒品危害防制條例第2條第2項第
    2款所列之第二級毒品並經公告列為禁藥管理，屬藥事法所
    稱之禁藥，依法不得轉讓，竟基於轉讓禁藥之犯意，於如附
    表二編號2至7所示之時間、地點、方式轉讓禁藥甲基安非他
    命與楊○○6次、蘇○○1次、黃○○1次、王○○1次、黃○
    ○2次。
二、案經福建金門地方法院檢察署檢察官簽分偵辦及金門縣警察
    局金城分局、金湖分局報告福建金門地

In [77]:
%run tool.ipynb
#test m1_
m1_ = m1[-2]
if '賺' in m1_:
    print('pass')
row = []
row.append(file)
#cat
for cat in cats:
    if cat in m1_:
        row.append(cat)
        break
#weis
wei = get_weight_g(m1_)
row.append(wei)
#dollar
if len(get_prices(m1_)) == 1: 
    print(1)
    dol = get_prices(m1_)[0]
else:
    if re.search('\W\w*元\w*[販賣|販售]\w*',m1_) != None: #取與販賣在同一句的金額
        print(2)
        dol = get_prices( re.findall('\W\w*元\w*[販賣|販售]\w*',m1_)[0] )
        dol = int(dol[0])
    else:
        prices = [int(x) for x in get_prices(m1_)]
        prices.sort()
        dol = prices[-1]
if re.search('(\w\w\w){}'.format(dol),m1_) != None:
    if re.search('(\w\w\w){}'.format(dol),m1_).group(1) == '每公克':
        dol = dol*wei
row.append(dol)
test.append(tuple(row) )
m1_,row,wei

1


('揭居所，將甲基安非他命半台兩即18.5公克販賣予顏玉玲，顏玉玲自其中取出約2點多公克（詳下述），留供自己施用外，餘16.5公克之甲基安非他命，依約以55000元之價格販售予王志宇牟利。嗣王志',
 ['D:/data/毒品/單被告判決書_3\\福建金門地方法院,101,訴,2,20120425,1.txt',
  '甲基安非他命',
  ['18.5', '2', '16.5', '55000'],
  '55000'],
 ['18.5', '2', '16.5', '55000'])

# -------------------------------------------------------

# (ing2)有table的檔案中無法被drug_trade抓出的用專屬表格的方式抓
先寫好抓出來的條件，再去看這些結果中有沒有錯的，那錯的又怎麼排除 \
(每個編號中的每個價錢要獨立抓出來存進list，在一個一個丟進convert_to_digits) \
1.寫好判斷編號中是否有 毒品名+元+公克 \
2.從1.篩出的編號看有無多個價錢 \
3.多價檔用drug_trade抓不出來的看如何處理 \
每個編號中有多個總價時用該行有'交易價格' 沒有'原審'的金額 \

In [51]:
###前置 存出有表格檔案的list
import re
tab= []
have= []
dont = []
for i,file in enumerate(files):
    if i % 1700 == 0:
        print(i,end=',')
    #name = one_accused(file)
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    txt1 = txt.replace('\n\n','\n')
    txt2 = txt.replace('\n','').replace('\u3000','').replace(' ','')
    if '\ntable\n' in txt:
        table = txt.split('\ntable\n')[1]
        tab.append(file)
    else:
        continue
    m1 = drug_trade(txt2)
    if m1 != []:
        have.append(file)
    else:
        dont.append(file)

0,1700,3400,5100,6800,8500,10200,11900,13600,15300,17000,

In [52]:
len(tab),len(have),len(dont)

(12016, 4496, 7520)

In [64]:
#1.寫好判斷編號中是否有 毒品名+元+公克
match = []
no_match = []
cats = drug_cats()
for i,file in enumerate(tab):
    if i % 1200 == 0:
        print(i,end=',')
    #file = tab[2]
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    #name = one_accused_txt(txt)
    table = txt.split('\ntable\n')[1]
    cases = table.split('###編號:')[1:]
    onoff = 0
    for case in cases:
        if onoff ==1:
            break
        if '元' in case and '公克' in case:
            for cat in cats:
                if cat in case:
                    match.append(file) #3901件
                    onoff = 1
                    break
    if onoff == 0: #無任何匹配的編號，有8115件
        no_match.append(file)
    #print(table)

0,1200,2400,3600,4800,6000,7200,8400,9600,10800,12000,

In [66]:
len(match),len(no_match)

(3901, 8115)

In [67]:
#2-1 從1.篩出的編號看有無多個價錢
ma_prices = []
ma_no = []
cats = drug_cats()
for i,file in enumerate(match):
    if i % 1200 == 0:
        print(i,end=',')
    #file = tab[2]
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    #name = one_accused_txt(txt)
    table = txt.split('\ntable\n')[1]
    cases = table.split('###編號:')[1:]
    onoff = 0
    for case in cases:
        if len(case.split('元')) > 2: #多個價錢:
            ma_prices.append(file) #3901件
            onoff = 1
            break
    if onoff == 0: #無任何匹配的編號，有8115件
        ma_no.append(file)

0,1200,2400,3600,

In [68]:
len(ma_prices),len(ma_no)

(3436, 465)

In [71]:
with open('D:/Google雲端硬碟/毒品code/table中有多個價.txt','w',encoding= 'utf-8-sig') as f:
    f.write('\n'.join(ma_prices) )

In [109]:
#3.多價檔用drug_trade抓不出來的看如何處理
ma_prices_yes =[]
ma_prices_no =[]
for i,file in enumerate(ma_prices):
    if i % 340 == 0:
        print(i,end=',')
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    txt1 = txt.replace('\n\n','\n')
    txt2 = txt.replace('\n','').replace('\u3000','').replace(' ','')
    content = '書記官'.join(txt2.split('書記官')[:-1])
    if '\ntable\n' in txt:
        table = txt.split('\ntable\n')[1]
    name = one_accused_txt(txt)
    result = drug_trade(table,name)
    if result == []:
        ma_prices_no.append(file)
    else:
        ma_prices_yes.append(file)

0,340,680,1020,1360,1700,2040,2380,2720,3060,3400,

In [113]:
len(ma_prices_no),len(ma_prices_yes)

(2153, 1283)

In [122]:
ex = '500元（未收訖）────甲基安非他命1包（0.3公克）'
drug_trade(ex,name)

[]

In [121]:
#3-2 從1.篩出的編號看有無多個價錢
file = ma_prices_no[100] #1000
with open(file,encoding='utf-8-sig') as f:
    txt =f.read()
txt1 = txt.replace('\n\n','\n')
txt2 = txt.replace('\n','').replace('\u3000','').replace(' ','')
content = '書記官'.join(txt2.split('書記官')[:-1])
if '\ntable\n' in txt:
    table = txt.split('\ntable\n')[1]
cases = table.split('###編號:')[1:]
#print(txt1,'-'*100)
for case in cases:
    if '元' in case and '公克' in case:
        for cat in cats:
            if cat in case:
                print(case)
                break

1
#交易時間────交易對象────交易地點:105年4月21日23時2分許────許亞傑────張瀚陽位於基隆市中正區新豐街303巷17弄23號2樓住處樓梯間
#交易方式:許亞傑持用行動電話門號0900000000號與張瀚陽持用之行動電話門號0000000000號聯繫，兩人相約見面後，張瀚陽於左列時、地，將右列毒品交予許亞傑，並收取右列價金。
#價格新臺幣────交易數量:500元（未收訖）────甲基安非他命1包（0.3公克）
#卷附相關證據:1.證人許亞傑之證言105年5月19日警詢（105年度偵字第2300號卷第38至40頁）105年5月19日偵訊（105年度偵字第2300號卷第182頁反面）2.通訊監察譯文（105年度偵字第2300號卷第42頁）3.被告之自白105年5月19日警詢（105年度偵字第2300號卷第13頁反面）105年5月19日偵訊（105年度偵字第2300號卷第161頁）105年6月20日偵訊（105年度偵字第2300號卷第210頁）105年7月11日本院訊問（本院105年度訴字第410號卷第16至18頁）105年7月27日準備程序、審理（本院105年度訴字第410號卷第56、63頁反面）
#主文（罪名、應處刑罰及沒收）:張瀚陽販賣第二級毒品，累犯，處有期徒刑壹年拾月。扣案如附表三所示之物均沒收。

2
#交易時間────交易對象────交易地點:105年4月9日16時2分許────蔡瑞濱────蔡瑞濱位於基隆市中正區新豐街370號13樓住處
#交易方式:蔡瑞濱持用行動電話門號0900000000號與張瀚陽持用之行動電話門號0000000000號聯繫，兩人相約見面後，張瀚陽於左列時、地，將右列毒品交予蔡瑞濱，並收取右列價金。
#價格新臺幣────交易數量:5000元（收訖）────甲基安非他命1包（8公克）
#卷附相關證據:1.證人蔡瑞濱之證言105年5月19日警詢（105年度偵字第2300號卷第27頁反面）105年5月19日偵訊（105年度偵字第2300號卷第184頁反面）2.通訊監察譯文（105年度偵字第2300號卷第31頁）3.被告之自白105年5月19日偵訊（105年度偵字第2300號卷第161頁）105年6月20日偵訊（105年度偵字第2300號卷第210頁）105年7月11日本院訊問（本院105年度訴字

In [46]:
import random
for i in range(10):
    num = random.randint(0,7520)
    file = dont[num]
    #name = one_accused(file)
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    txt1 = txt.replace('\n\n','\n')
    txt2 = txt.replace('\n','').replace('\u3000','').replace(' ','')
    content = '書記官'.join(txt2.split('書記官')[:-1])
    if '\ntable\n' in txt:
        table = txt.split('\ntable\n')[1]
    cases = table.split('###編號:')[1:]
    for case in cases:
        if len(case.split('元')) > 2: #多個價錢
            print(case)
            break
    '''
    keys = ['#購毒者','#購買者']
    onoff = '需看檔'
    for key in keys:
        if key in table:
            onoff = '有交易'
            break
    if onoff =='需看檔':
        print(num,table)
    '''

1
#交易聯絡之對象、方式、時間、地點:邱俊維於102年12月27日5時18分許、5時19分許、9時54分許、10時56分許，11時40分許、12時12分許、12時40分許、12時45分許、12時59分許以其使用之0000000000號行動電話與曾文使用之0000000000號行動電話聯絡後，隨即在曾文位於宜蘭縣羅東鎮○○路000巷0號2樓之住處附近見面，曾文以500元之代價販賣第一級毒品海洛因給邱俊維。
#販毒所得（新臺幣）:500元
#主文:曾文販賣第一級毒品，處有期徒刑柒年捌月，未扣案之行動電話壹支（含門號0000000000號SIM卡壹張）沒收之，如全部或一部不能沒收時，追徵其價額；未扣案之販賣第一級毒品所得新臺幣伍佰元，沒收之，如全部或一部不能沒收時，以其財產抵償之。

1
#對象姓名:黃証約
#被告黃全興販賣海洛因時間、地點:102年5月14日上午10時45分許，黃証約以門號0000000000號行動電話跟被告使用之門號0000000000號行動電話聯絡過後，於同日上午11時10分許，在嘉義縣鹿草鄉三角村八掌溪（起訴書誤載為急水溪）堤防邊，向被告購買500元之海洛因1包。
#論罪科刑:黃全興販賣第一級毒品，累犯，處有期徒刑柒年拾月。扣案之門號0000000000號行動電話壹支（含SIM卡壹張）沒收；未扣案販賣第一級毒品所得新臺幣伍佰元沒收，如全部或一部不能沒收時，以其財產抵償之。
#2#

1
#購毒者:洪宗祺
#交易時間:105年4月7日17時32分許聯絡完畢之同日某時許
#交易地點:南投市文化路102巷7號（張東林租屋處）
#毒品種類/金額（新臺幣）:1包海洛因（約1/8錢）/價格3,000元
#交易方式:張東林以持用之門號0000000000號行動電話附設之通訊軟體LINE與洪宗祺持用之門號0985853566號行動電話附設之通訊軟體LINE聯絡後，兩人在左列地點碰面，洪宗祺交付3,000元給張東林，張東林交付左列毒品給洪宗祺，而完成交易。
#主文:張東林販賣第一級毒品，累犯，處有期徒刑柒年捌月。未扣案不詳廠牌行動電話壹具（含門號0000000000號SIM卡壹枚）及犯罪所得新臺幣參仟元沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。

1
#交易時間:104年9月22日10時55分許
#交易地點:臺南市○○區○○路郵局前
#交易對象:林佑

In [43]:
cases[1].split('元')

['2\n#交易時間:106年9月20日凌晨4時許\n#交易地點:臺南市○區○○街00號富地飯店前\n#交易對象:楊福順\n#交易數量:甲基安非他命1包（重約19公克）\n#交易價格即本院認定被告本件犯罪所得:1萬5千',
 '\n#原審認定被告本件犯罪所得:3千',
 '\n#原審諭知之罪名及宣告刑:吳修銘共同販賣第二級毒品，累犯，處有期徒刑肆年，扣案門號○○○○○○○○○○號行動電話壹支（含SIM卡壹張）沒收，未扣案犯罪所得新臺幣參仟',
 '沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。\n#本院諭知之罪名及宣告刑:吳修銘販賣第二級毒品，處有期徒刑壹年陸月，扣案門號○○○○○○○○○○號行動電話壹支（含SIM卡壹張）沒收，未扣案犯罪所得新臺幣壹萬伍仟',
 '沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。\n']

# ------------------------------------------------------------------------

In [105]:
oth = []
notin = []
cats = drug_cats()[1:]
for i,file in enumerate(files):
    if i % 1700 == 0:
        print(i,end=',')
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    txt1 = txt.replace('\n\n','\n')
    txt2 = txt.replace('\n','').replace('\u3000','').replace(' ','')
    if drug_cats()[0] not in txt2:
        notin.append(file)
    for cat in cats:
        if cat in txt2:
            oth.append([file,cat])
            break
            

0,1700,3400,5100,6800,8500,10200,11900,13600,15300,17000,